In [45]:
import pandas as pd
import datetime as dt
import numpy as np
import warnings
import json
import re
import sys
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import matplotlib.pyplot as plt
import seaborn as sns
sys.path.append('/Users/alenastern/Documents/Spring2018/Machine_Learning/Machine_Learning_Public_Policy/hws/hw2')
import explore as ex
import string
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
warnings.filterwarnings('ignore')

In [6]:
json_file = '../data/cities.json'

with open(json_file, 'r') as f:
    md = json.load(f)

In [52]:
data = pd.DataFrame(columns = ['city', 'month_year', 'Summary'])
city_list = []
for key, value in md.items():
    city = value['name']
    filepath = '/Users/alenastern/Google Drive File Stream/My Drive/Alena_Project/PR_Data/{}.csv'.format(city)
    if value["desc"] == "Y":
        try:
            df = pd.read_csv(filepath)
        except:
            try:
                df = pd.read_csv(filepath, encoding='mac_roman')
            except:
                continue
        print(key)
        name = key.split(' ')
        city_list.append(name[0].lower())
    else:
        continue
    
    try:
        df['Create Date'] = pd.to_datetime(df['Create Date'])
    except:
        df['New'] = pd.to_datetime(df['Create Date'].apply(lambda x: re.findall('^\S*', x)[0]))
        df.drop(columns=['Create Date'], inplace = True)
        df.rename(index=str, columns={"New": "Create Date"}, inplace = True)

    df['month_year'] = df['Create Date'].dt.to_period('M')
    
    mc = df[['month_year', 'Summary']]
    mc['city'] = city
    
    data = pd.concat([data, mc])

Arlington city
Asheville city
Bainbridge Island city
Boulder County
Cathedral City city
Dayton city
Denton city
Everett city
Fort Collins city
Greensboro city
Hayward city
Kirkland city
Las Cruces city
Lynnwood city
Mercer Island city
Miami city
Middleborough town
New Orleans city
Oakland city
Oklahoma City city
Olympia city
Palo Alto city
Peoria city
Pullman city
Rancho Cucamonga city
Redmond city
Renton city
Sacramento city
San Francisco city
Tukwila city
Vallejo city
West Sacramento city
Winchester city


In [43]:
data.shape

(86416, 3)

In [11]:
len(data.city.unique())

33

In [12]:
data.head()

,Summary,city,month_year
0,We are working with an engineering firm on an ...,Arlington,2018-06
1,Need copies of contracts and all related docum...,Arlington,2018-06
2,"Copies of Building Permits of $5,000 valuation...",Arlington,2018-06
3,police report filed to an officer against Wayn...,Arlington,2018-06
4,"Email Communications between Stephanie Shook, ...",Arlington,2018-06


In [15]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [53]:
# Turn to lowercase
data.Summary = data.Summary.str.lower()

# Remove all punctuation
translator = str.maketrans('','', string.punctuation)
data.Summary = data.Summary.str.translate(translator)

# Remove all city names
for city in city_list:
    data.Summary = data.Summary.str.replace(city, '')
    
# Remove whitespace
#wsp_translator = str.maketrans(' ',' ', string.whitespace)
#data.Summary = data.Summary.str.translate(wsp_translator)

# Remove digits
dig_translator = str.maketrans('','', string.digits)
data.Summary = data.Summary.str.translate(dig_translator)

#split on whitespace
data.Summary = data.Summary.str.split(' ')

#remove empty strings, stopwords and stem
stop_words = set(stopwords.words('english'))
ps = PorterStemmer()
data.mash = data['Summary'].apply(lambda x: i for i in x if len(i) > 0 and i not in stop_words)
#[ps.stem(i) for i in data.Summary if len(i) > 0 and i not in stop_words]

## Remove stop words
#data.Summary = [i for i in data.Summary if i not in stop_words]

# Stem words
#stemmer = PorterStemmer

# Remove very rare words

# Remove words that are very common to requests (file, copy, document, request)

TypeError: unhashable type: 'list'

In [54]:
data.head()

,Summary,city,month_year
0,"[we, are, working, with, an, engineering, firm...",Arlington,2018-06
1,"[need, copies, of, contracts, and, all, relate...",Arlington,2018-06
2,"[copies, of, building, permits, of, , valuatio...",Arlington,2018-06
3,"[police, report, filed, to, an, officer, again...",Arlington,2018-06
4,"[email, communications, between, stephanie, sh...",Arlington,2018-06


In [59]:
data['mash'] = data['Summary'].apply(lambda x: [i for i in x if len(i) > 0 and i not in stop_words])

TypeError: 'float' object is not iterable

In [35]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml
showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [60]:
data.head()

,Summary,city,month_year
0,"[we, are, working, with, an, engineering, firm...",Arlington,2018-06
1,"[need, copies, of, contracts, and, all, relate...",Arlington,2018-06
2,"[copies, of, building, permits, of, , valuatio...",Arlington,2018-06
3,"[police, report, filed, to, an, officer, again...",Arlington,2018-06
4,"[email, communications, between, stephanie, sh...",Arlington,2018-06


In [62]:
data[isinstance(data['Summary'], float)]

KeyError: False